In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-28 21:55:27--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2021-10-28 21:55:27 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [7]:
# Filter by price on certain columns
vine_df.filter("total_votes=20").select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1RTIKR8HWSJYQ|          1|            0|         20|   N|                N|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|
| RYJVJ765QAQJW|          1|           14|         20|   N|                N|
|R18YE5XIUB40P9|          5|           18|         20|   N|                Y|
|R18UIVCYVG6HKQ|          3|           13|         20|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
helpful_filter_df = vine_df.filter("helpful_votes/total_votes >= .50")
helpful_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23H79DHOZTYAU|          1|            1|          1|   N|                Y|
|R3AACSKX1EAHUY|          5|            1|          1|   N|                Y|
|R1FJWU0IJ4KUT4|          1|            5|          5|   N|                Y|
|R23HC6NMA3HBPS|          5|            1|          1|   N|                N|
| R8HD4KOH8L0A6|          5|            1|          2|   N|                Y|
|R3DCTXDTB028VH|          5|            3|          4|   N|                Y|
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2NCZ6CIHAWH28|          5|            1|          1|   N|                Y|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RXJ4RXVZNQQMK|          1|            1|          1|   N|     

In [9]:
vine_paid_df = helpful_filter_df.filter(helpful_filter_df["vine"] == "Y")
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RWKQQPG100CQ4|          4|            2|          4|   Y|                N|
|R1R0J8S1VGALUY|          1|            1|          2|   Y|                N|
|R28165EG0N9CSN|          4|            1|          2|   Y|                N|
|R1CH0ZXGIMVEFC|          5|            1|          2|   Y|                N|
|R2BFEJHEKH4YWZ|          5|            3|          4|   Y|                N|
| RYIG0BYEZP7WM|          2|            1|          1|   Y|                N|
|R20FZQEZM1277P|          4|            2|          4|   Y|                N|
| RSAY2UB0IHC0L|          4|            4|          6|   Y|                N|
| RQKQVDDOKW5MU|          5|            4|          5|   Y|                N|
|R379LKZSA2PFOZ|          4|            1|          2|   Y|     

In [10]:
vine_paid_review_count = vine_paid_df.count()
print(vine_paid_review_count)

1757


In [11]:
vine_unpaid_df = helpful_filter_df.filter(helpful_filter_df["vine"] == "N")
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R23H79DHOZTYAU|          1|            1|          1|   N|                Y|
|R3AACSKX1EAHUY|          5|            1|          1|   N|                Y|
|R1FJWU0IJ4KUT4|          1|            5|          5|   N|                Y|
|R23HC6NMA3HBPS|          5|            1|          1|   N|                N|
| R8HD4KOH8L0A6|          5|            1|          2|   N|                Y|
|R3DCTXDTB028VH|          5|            3|          4|   N|                Y|
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2NCZ6CIHAWH28|          5|            1|          1|   N|                Y|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RXJ4RXVZNQQMK|          1|            1|          1|   N|     

In [12]:
vine_unpaid_review_count = vine_unpaid_df.count()
print(vine_unpaid_review_count)

572411


In [13]:
vine_unpaid_5star = vine_unpaid_df.filter(vine_unpaid_df["star_rating"] == "5").count()
print(vine_unpaid_5star)


291135


In [14]:
vine_paid_5star = vine_paid_df.filter(vine_paid_df["star_rating"] == "5").count()
print(vine_paid_5star)

700


In [15]:
vine_unpaid_5star_percent = (vine_unpaid_5star / vine_unpaid_review_count)
print(vine_unpaid_5star_percent)

0.5086118191299608


In [16]:
vine_paid_5star_percent = (vine_paid_5star / vine_paid_review_count)
print(vine_paid_5star_percent)

0.398406374501992


In [17]:
total_review_count = (vine_paid_review_count + vine_unpaid_review_count)
print(total_review_count)

574168


In [19]:
print("Total number of reviews: %f" % total_review_count)
print("Total UNPAID reviews: %f" % vine_unpaid_review_count)
print("Total UNPAID 5 star reviews: %f" % vine_unpaid_5star)
print("Percent UNPAID 5 star reviews: %f" % vine_unpaid_5star_percent)
print("Total PAID reviews: %f" % vine_paid_review_count) 
print("Total PAID 5 star reviews: %f" % vine_paid_5star)
print("Percent PAID 5 star reviews: %f" % vine_paid_5star_percent)

Total number of reviews: 574168.000000
Total UNPAID reviews: 572411.000000
Total UNPAID 5 star reviews: 291135.000000
Percent UNPAID 5 star reviews: 0.508612
Total PAID reviews: 1757.000000
Total PAID 5 star reviews: 700.000000
Percent PAID 5 star reviews: 0.398406
